In [1]:
import pandas as pd
import selenium
import webdriver_manager
from datetime import datetime 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
import pandas as pd
from selenium import webdriver

from tqdm import tqdm
import os
from bs4 import BeautifulSoup
import requests
import re

import zipfile
import shutil

import os


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import re

pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

import json
# Ignorar warings
import warnings
warnings.filterwarnings("ignore")

import dotenv
from dotenv import load_dotenv
load_dotenv()

import psycopg2
from psycopg2 import sql
pd.set_option('display.max_columns', None)

import sys
sys.path.append("../")  

In [ ]:
keywallapop= os.getenv("keywallapop")

Cuidado, de wallapop solo hay 20 llamadas mensuales

In [ ]:

url = "https://wallapop3.p.rapidapi.com/search"

querystring = {"query":"nissan","minPrice":"2000","sort":"mostRecent"}

headers = {
	"x-rapidapi-key": keywallapop,
	"x-rapidapi-host": "wallapop3.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

resposa=response.json()

SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit)

In [ ]:
# with open("../src", "w") as json_file:
#     json.dump(resposa, json_file, indent=4)

In [ ]:
len (resposa)

In [ ]:
lista=[]
df_final=pd.DataFrame()
for i in range(0,40):
    id_wallapop=resposa[i]["id"]
    id_user=resposa[i]["user_id"]
    version=resposa[i]["title"]
    descripcion=resposa[i]["description"]
    id_categoria=resposa[i]["category_id"]
    precio=resposa[i]["price"]["amount"]
    latitud=resposa[i]["location"]["latitude"]
    longitud=resposa[i]["location"]["longitude"]
    ciudad=resposa[i]["location"]["city"]
    provincia=resposa[i]["location"]["region2"]
    envio_disponible=resposa[i]["shipping"]["item_is_shippable"]
    reparado=resposa[i]["is_refurbished"]["flag"]

    lista.append([id_wallapop,id_user,title,descripcion,id_category,precio,latitud, longitud,ciudad,provincia,envio_disponible,reparado] )
    df=pd.DataFrame(lista)
    df_final= pd.concat([df_final,df])

In [ ]:

dicc_encabezados={0:"id_wallapop",
                  1: "id_user",
                  2: "version",
                  3: "descripcion",
                  4: "id_categoria",
                  5: "precio",
                  6: "latitud",
                  7: "longitud",
                  8: "ciudad",
                  9: "provincia",
                  10: "envio_disponible",
                  11: "reparado"}
df_final.columns= df_final.columns.map(dicc_encabezados)

In [ ]:
df_final.sample(3)

In [ ]:
df_final.shape

ESTA LISTO PARA LA LIMPIEZA LO DE WALLAPOP

**Tratamiento de duplicados y nulos Wallapop**  
*NULOS*  

No hay nulos

*DUPLICADOS*  

Me he fijado que tiene muchos duplicados y que por alguna razon en la extraccion se han repetido con lo que sería necesario eliminarlos (o los users han subido el mismo anuncio muchas veces cosa que dudo porque wallapop no deja)

*DATOS INNECESARIOS*  

Eliminamos las columnas que no nos interesan y que aportan más problemas que información   

He intentado quedarme con el nombre de la version en una nueva columna pero hay demasiados anuncios que no ponen el nombre de la version entonces no me valía.


*Importante a tener en cuenta*  

Al final me quedo solo con la provincia de Madrid porque el resto del proyecto lo estoy haciendo en base a Madrid provincia


SOLO NOS INTESESA:  
- id (pasaré a numérico)  
- user_id  
- title  
- description  
- category_id (lo quiero para filtrar por 100(codigo correspondiente a coches)para que no me coja otros códigos que son de carroceria etc)  
- price (que está en euros)  
- location (por si las moscas)  
- capacidad de traslado (no creo que me lo quede)  
- refurbished(igual la mantengo pero los true los paso a 1 y los false a 0 porque en sql existe el bit pero no tiene valores true)

In [ ]:
df_final.duplicated().sum()

In [ ]:
df_final.drop_duplicates(inplace=True)

Ahora que ya he comprobado que solo hay categoría coche, la elimino

In [ ]:
df_final["id_categoria"].unique()

In [ ]:
df_final["version"].unique()

In [ ]:
df_final["envio_disponible"].unique()

In [ ]:
df_final["reparado"].unique()

In [ ]:
df_final=df_final.drop(columns= ["id_wallapop","id_user","id_categoria", "version", "envio_disponible", "reparado"])

In [ ]:
df_final.info()

In [ ]:
df_final.sample(3)

In [ ]:
df_final=df_final[df_final["provincia"]=="Madrid"]
df_final.reset_index(drop=True, inplace=True)

Añado un id

In [ ]:
df_final.index=df_final.index+1
df_final.reset_index(inplace= True)

In [ ]:

dicc_encabezados0={"index":"id_wallapop",
                  "descripcion": "descripcion",
                  "precio": "precio",
                  "latitud": "latitud",
                  "longitud": "longitud",
                  "ciudad": "ciudad",
                  "provincia": "provincia"}

df_final.columns=df_final.columns.map(dicc_encabezados0)

In [ ]:
df_final.to_csv("../datos/datos_wallapop.csv")
df_wallapop=pd.read_csv("../datos/datos_wallapop.csv", index_col= "Unnamed: 0" )

In [10]:
df_wallapop.head(4)

,id_wallapop,descripcion,precio,latitud,longitud,ciudad,provincia
0,1,nisan almera tino del 2004 ba por cadena le ve...,2200,40.301646,-3.441828,Arganda del Rey,Madrid
1,2,vendo mi coche en perfecto estado de motor y d...,6500,40.620256,-3.910636,Hoyo de Manzanares,Madrid
2,3,coche usado por mi mujer para ir a trabajar.\n...,11500,40.494190,-3.961537,Villanueva del Pardillo,Madrid
3,4,"se vende nissan patrol k260 embrague nuevo , b...",4500,40.419014,-3.707443,Madrid,Madrid


API seguridad del modelo

In [ ]:
url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/2013/make/nissan"
response3 = requests.get(url_anio)
if response3.status_code == 200:
    modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
else:
    print(f"Error: {response3.status_code} - {response3.text}")
    


In [ ]:


df_ratings = pd.DataFrame()

for anio in range(2011, 2026):
    url_anio = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan"
    response3 = requests.get(url_anio)
    
    #verifica si no da error
    if response3.status_code == 200:
        try:
            modelos = [diccio["Model"] for diccio in response3.json().get("Results", [])]
        except (KeyError, ValueError) as e:
            print(f"Error al obtener modelos para el año {anio}: {e}")
            continue
    else:
        print(f"Error en la solicitud para el año {anio}: {response3.status_code}")
        continue

    modelos_ids = []
    for modelo in modelos:
        url3 = f"https://api.nhtsa.gov/SafetyRatings/modelyear/{anio}/make/nissan/model/{modelo}"
        response = requests.get(url3)

        if response.status_code == 200:
            try:
                modelos_ids.extend([diccio["VehicleId"] for diccio in response.json().get("Results", [])])
            except (KeyError, ValueError) as e:
                print(f"Error al obtener VehicleId para modelo {modelo} en el año {anio}: {e}")
                continue
        else:
            print(f"Error en la solicitud para modelo {modelo}, año {anio}: {response.status_code}")
            continue

    #conseguir ratings con modelos_ids
    for modelo_id in modelos_ids:
        url2 = f"https://api.nhtsa.gov/SafetyRatings/VehicleId/{modelo_id}"
        response = requests.get(url2)

        if response.status_code == 200:
            try:
                df = pd.DataFrame(response.json().get("Results", []))
                df_ratings = pd.concat([df_ratings, df], ignore_index=True)
            except (KeyError, ValueError) as e:
                print(f"Error al obtener ratings para VehicleId {modelo_id}: {e}")
        else:
            print(f"Error en la solicitud para VehicleId {modelo_id}: {response.status_code}")


In [ ]:
df_ratings.sample(4)

In [ ]:
df_ratings.info()

In [ ]:
df_ratings.to_csv("../datos/datos_seguridad.csv")
df_seguridad=pd.read_csv("../datos/datos_seguridad.csv", index_col= "Unnamed: 0" )

ESTÁ listo para la limpieza lo de SEGURIDAD

**Tratamiento de duplicados y nulos Seguridad**  
*NULOS*  

Había muchos Not Rated que convertí en nulos para eliminarlos porque si no tiene rating, no nos aporta nada de valor 

*DUPLICADOS*  

No tiene duplicados


*DATOS INNECESARIOS*  

Eliminamos las columnas que no nos interesan y que aportan más problemas que información 


In [ ]:
df_seguridad=df_seguridad.drop(columns= ["VehiclePicture","FrontCrashDriversideRating","FrontCrashPassengersideRating", "SideCrashDriversideRating", "SideCrashPassengersideRating", "combinedSideBarrierAndPoleRating-Front", "combinedSideBarrierAndPoleRating-Rear", "RolloverRating2", "RolloverPossibility2", "dynamicTipResult", "FrontCrashPicture", "FrontCrashVideo", "SideCrashPicture", "SideCrashVideo", "SidePolePicture", "SidePoleVideo"])

In [ ]:
df_seguridad.sample(4)

In [ ]:
df_seguridad["estabilidad_electronica"].value_counts()

In [ ]:
df_seguridad["aviso_choque_frontal"].value_counts()

In [ ]:
df_seguridad["aviso_desvio_carril"].value_counts()

In [ ]:
df_seguridad.replace("Not Rated", pd.NA, inplace=True)

columnas_rating = ["OverallRating", "OverallFrontCrashRating", "OverallSideCrashRating","sideBarrierRating-Overall", "RolloverRating", "SidePoleCrashRating"]
df_seguridad.dropna(subset=columnas_rating, inplace=True)
df_seguridad.reset_index(drop= True, inplace=True)

In [ ]:
df_seguridad[columnas_rating] = df_seguridad[columnas_rating].astype(int)

In [ ]:
df_seguridad.duplicated().sum()

In [ ]:
df_seguridad.info()

Añado un id

In [ ]:
df_seguridad.index= df_seguridad.index+1
df_seguridad.reset_index(inplace=True)

In [ ]:
dicc_encabezados={
    "index": "id_seguridad",
    "OverallRating": "evaluacion_general",
    "OverallFrontCrashRating": "evaluacion_choque_frontal",
    "OverallSideCrashRating": "evaluacion_choque_lateral",
    "sideBarrierRating-Overall": "evaluacion_barrera_lateral",
    "RolloverRating": "evaluacion_vuelco",
    "RolloverPossibility": "probabilidad_vuelco",
    "SidePoleCrashRating": "evaluacion_choque_columna",
    "NHTSAElectronicStabilityControl": "estabilidad_electronica",
    "NHTSAForwardCollisionWarning": "aviso_choque_frontal",
    "NHTSALaneDepartureWarning": "aviso_desvio_carril",
    "ComplaintsCount": "quejas",
    "RecallsCount": "retiradas",
    "InvestigationCount": "investigaciones",
    "ModelYear": "anio",
    "Make": "marca",
    "Model": "version",
    "VehicleDescription": "descripcion",
    "VehicleId": "vehiculo_id"
}
            
df_seguridad.columns=df_seguridad.columns.map(dicc_encabezados)

In [ ]:
df_seguridad.to_csv("../datos/datos_seguridad.csv")

Empezamos con web scrapping de Quecochemecompro

Este funciona sin duplicados(Primera tabla solo porque es donde están los modelos más simples y el cliente al que estamos intentando informar prioriza mucho el ahorro)

In [ ]:
url_nuevo = "https://www.quecochemecompro.com/"

driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_nuevo)

#acepto cookies
driver.find_element("css selector", "#qc-cmp2-ui > div.qc-cmp2-footer.qc-cmp2-footer-overlay.qc-cmp2-footer-scrolled > div > button.css-y6krop").click()
sleep(2)
#me meto en nuevos
driver.find_element("css selector", "#header-menu > li:nth-child(1) > a").click()


sleep(2)
#click en el buscador 
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form").click()
sleep(2)
#meto nissan en el buscador
driver.find_element("css selector", "#header > div.tw-container.flex.gap-4 > div > div.hidden.lg\:flex.w-60.ml-auto.mr-8.gap-4.items-center > form > input").send_keys("nissan", Keys.ENTER)

#saco los nombres para meterlos en una lista

sopa_nuevos = BeautifulSoup(driver.page_source, 'html.parser')
lista_nombre_modelo=[]
for modelo in sopa_nuevos.find_all("li", {"class":"mx-auto w-full min-w-[20rem]"}):
    tag_a = modelo.find("a", title=True)
    if tag_a:
        lista_nombre_modelo.append(tag_a["title"])


df_datos_modelos = pd.DataFrame()
for n_modelo in range(1, len(lista_nombre_modelo) + 1):
    sleep(2)
    # Hacer click en el modelo específico
    driver.find_element("css selector", f"#results > ul > li:nth-child({n_modelo}) > div > div.relative.z-10 > div.flex.justify-between.items-end.py-3.leading-5.text-17 > div:nth-child(1) > a").click()
    sopa_tabla = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(2)
    df_tablas=pd.DataFrame()
    
    try:  # Encontrar la tabla de datos
        
        sleep(2) #hace click por si hay más tablas

        try:
            driver.find_element("css selector", f"#versions > div > div:nth-child(2) > table > tfoot > tr > td > ul > li:nth-child(2) > a").click()
            
        except:
            # Si no encuentra el selector con nth-child(2), intenta con nth-child(3)
            
            driver.find_element("css selector", f"#versions > div > div:nth-child(3) > table > tfoot > tr > td > ul > li:nth-child(2) > a").click()
            
        tabla = sopa_tabla.find("table")  #a lo mejor no me coge en algunas porque hay otra tabla                
        
        encabezados = [th.get_text(strip=True) for th in tabla.find_all("thead")[0].find_all("th")]

        filas = []
        for tr in tabla.find_all("tbody")[0].find_all("tr"):
            filas.append([td.get_text(strip=True) for td in tr.find_all("td")])

        df_tabla = pd.DataFrame(filas, columns=encabezados)
        df_tabla["Modelo"] = lista_nombre_modelo[n_modelo - 1]
        df_tablas= pd.concat([df_tablas, df_tabla])




        df_datos_modelos= pd.concat([df_datos_modelos, df_tablas], ignore_index=True)

    except:
        
        driver.back()
        continue
        
    driver.back()    




In [ ]:
df_datos_modelos.sample(3)

In [ ]:
df_datos_modelos.to_csv("../datos/datos_modelos_nuevos.csv")
df_datos_modelos_nuevos=pd.read_csv("../datos/datos_modelos_nuevos.csv", index_col= "Unnamed: 0" )

Coches nuevos listos para limpiar

**Tratamiento de duplicados y nulos Nuevos**  
*NULOS*  

Hay nulos en consumo y autonomía asique decido meter los valores de autonomia en los nan de consumo (porque corresponden a los coches electricos) y elimino la columna autonomía.  

Finalmente nos quedamos sin ningun nulo sin necesidad de hacer un dropna

*DUPLICADOS*  

No tiene duplicados


*DATOS INNECESARIOS*  

También elimino la columna etiqueta y oferta ya que no se ofrecen información relevante  

En la columna de Consumo me he fijado que hay 5 valores con "-" lo que no me permite formatear esa columna(dado que son tan pocos, voy a eliminarlos porque al tener la autonomía en la misma columna la media no es representativa)

In [ ]:
df_datos_modelos_nuevos.sample(5)

In [ ]:
df_datos_modelos_nuevos["Consumo"] = df_datos_modelos_nuevos["Consumo"].fillna(df_datos_modelos_nuevos["Autonomía"])


In [ ]:
df_datos_modelos_nuevos=df_datos_modelos_nuevos.drop(columns= ["Etiqueta","Oferta","Autonomía"])

In [ ]:
df_datos_modelos_nuevos["Consumo"]=df_datos_modelos_nuevos["Consumo"].str.replace("l/100Km", "").str.replace(",",".").str.replace(" Km", "")
df_datos_modelos_nuevos["PVP"]=df_datos_modelos_nuevos["PVP"].str.replace("€", "").str.replace(".", "")

In [ ]:
df_datos_modelos_nuevos.duplicated().sum()

In [ ]:
df_datos_modelos_nuevos=df_datos_modelos_nuevos[df_datos_modelos_nuevos["Consumo"]!= "-"]

In [ ]:
df_datos_modelos_nuevos["PVP"]=df_datos_modelos_nuevos["PVP"].astype(int)
df_datos_modelos_nuevos["Consumo"]=df_datos_modelos_nuevos["Consumo"].astype(float)

Añado un id

In [ ]:
df_datos_modelos_nuevos.index= df_datos_modelos_nuevos.index+1
df_datos_modelos_nuevos.reset_index(inplace=True)

In [ ]:
dicc_encabezados1={"index":"id_modelo_nuevo",
                    "Versión":"version",
                    "Puertas": "puertas",
                    "CV": "potencia",
                    "Consumo": "consumo",
                    "Motor": "motor",
                    "PVP": "precio",
                    "Modelo": "modelo"
                  }
            
df_datos_modelos_nuevos.columns=df_datos_modelos_nuevos.columns.map(dicc_encabezados1)

In [11]:
df_datos_modelos_nuevos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 0 to 106
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_modelo_nuevo  107 non-null    int64  
 1   version          107 non-null    object 
 2   puertas          107 non-null    int64  
 3   potencia         107 non-null    int64  
 4   consumo          107 non-null    float64
 5   motor            107 non-null    object 
 6   precio           107 non-null    int64  
 7   modelo           107 non-null    object 
 8   provincia        107 non-null    object 
 9   anio             107 non-null    int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 9.2+ KB


In [19]:
df_datos_modelos_nuevos["provincia"]= "Madrid"
df_datos_modelos_nuevos["anio"]= 2024

In [21]:
df_datos_modelos_nuevos.to_csv("../datos/datos_modelos_nuevos.csv")

Datos de modelos nuevos ya están formateados

Alternativa 2º mano

In [ ]:
url_alterantiva= "https://www.autoscout24.es/"
driver=webdriver.Chrome()
driver.maximize_window()
driver.get(url_alterantiva)

#acepto cookies
driver.find_element("css selector", "#as24-cmp-popup > div > div._acceptance-buttons_1lphq_85 > button._consent-accept_1lphq_114").click()

#despliego menu
driver.find_element("css selector", "#make").click()
sleep(2)
#elijo nissan
driver.find_element("css selector", "#make > optgroup:nth-child(3) > option:nth-child(172)").click()
sleep(2)
#le da a buscar
driver.find_element("css selector", "#search-mask-search-cta").click()
sleep(2)

#que nos de solo madrid
driver.find_element("css selector", "#__next > div > div > div.ListPage_wrapper__vFmTi > div.ListPage_container__Optya > div:nth-child(2) > aside > div > div:nth-child(2) > div:nth-child(1) > div > div.Location_wrapper__mvayy > div:nth-child(2) > div > div > div.input-wrapper > div > button > svg").click()
sleep(2)


df_completo=pd.DataFrame()
for pag in range(1,21):
    df=pd.DataFrame()
    url_pag=f"https://www.autoscout24.es/lst/nissan?atype=C&cy=E&damaged_listing=exclude&desc=0&lat=40.435712&lon=-3.6405248&page={pag}&powertype=kw&search_id=1yf3g5xh5i1&sort=standard&source=listpage_pagination&ustate=N%2CU&zipr=200"
    driver.get(url_pag)
    sopa_alternativa = BeautifulSoup(driver.page_source, 'html.parser')
    sleep(2)
    #conseguir nombre
    lista_nombres = sopa_alternativa.findAll("h2")
    todos_nombres=[nombre.getText().replace("\xa0", " ") for nombre in lista_nombres]
    df_nombre=pd.DataFrame(todos_nombres)


    #conseguir datos
    lista_datos= sopa_alternativa.findAll("div", {"class":"VehicleDetailTable_container__XhfV1"})
    todos_datos = [re.sub(r'(Manual|Automático)\d{2}/(\d{4})(Diésel|Gasolina|Electro/Gasolina|Eléctrico)\d*', r'\1 \2 \3', dato.getText()).replace("km", " ").replace("CV", "").replace("kW", "").replace(".", "").replace("(", "").replace(")", "").split() for dato in lista_datos]
    df_dato=pd.DataFrame(todos_datos)


    #Conseguir precio
    lista_precios= sopa_alternativa.findAll("p", {"class":"Price_price__APlgs PriceAndSeals_current_price__ykUpx"})
    todos_precios=[precio.getText().replace("€ ","").replace(",-","").replace(".","") for precio in lista_precios]
    df_precio=pd.DataFrame(todos_precios)

    df= pd.concat([df,df_nombre,df_dato,df_precio], axis=1, ignore_index=True)
    df_completo=pd.concat([df_completo, df], axis=0, ignore_index=True)


In [ ]:
df_completo.sample(5)

**Tratamiento de duplicados y nulos Segunda Mano**  
*NULOS*  

Hay nulos porque se crean algunas columnas por datos que están formateados distinto. Puedo permitirme eliminar las columnas casi llenas de nulos y luego eliminar los restantes para que los 367 que se queden, tengan datos.  

*DUPLICADOS*  

Decido no eliminarlos porque el hecho de que se repitan nos muestra que coches se están vendiendo más


*DATOS INNECESARIOS*  

En la columna de precio se cuela Ocasión como si fuera un precio lo que me dificultaría pasar al formato correcto por ello como apenas son 10 los elimino

In [ ]:
df_completo.info()

In [ ]:
df_completo.duplicated().sum()

In [ ]:
df_completo[df_completo.duplicated()].head()

In [ ]:
df_completo=df_completo.drop(columns= [7,8,9,10])

In [ ]:
df_completo.dropna(inplace=True)

In [ ]:
df_completo.info()

In [ ]:
dicc_encabezados2={0:"title",
                  1: "kilometraje",
                  2: "tipo",
                  3: "anio",
                  4: "fuel",
                  5: "potencia",
                  6: "precio"}
df_completo.columns=df_completo.columns.map(dicc_encabezados2)

In [ ]:
df_completo.sample(3)

Limpiamos los valores que no correspondan (En precio se han colado ocasion )

In [ ]:

df_completo = df_completo[~df_completo["precio"].str.contains("-/-Ocasión", na=False)]


In [ ]:
# df_completo["precio"].unique() Lo comento para que no moleste

In [ ]:
df_completo.info()

In [ ]:
df_completo.to_csv("../datos/datos_segunda_mano.csv")
df_segunda_mano=pd.read_csv("../datos/datos_segunda_mano.csv", index_col= "Unnamed: 0")

Aquí formateamos

In [ ]:
df_segunda_mano["kilometraje"]=df_segunda_mano["kilometraje"].astype(int)
df_segunda_mano["potencia"]=df_segunda_mano["potencia"].astype(int)
df_segunda_mano["precio"]=df_segunda_mano["precio"].astype(int)
df_segunda_mano["anio"]=df_segunda_mano["anio"].astype(int)


In [ ]:
df_segunda_mano.info()

Añado un id

In [ ]:
df_segunda_mano.index= df_segunda_mano.index+1
df_segunda_mano.reset_index(inplace=True)

In [ ]:
dicc_encabezados3={"index":"id_segunda_mano",
                    "title":"version",
                    "kilometraje": "kilometraje",
                    "tipo": "tipo",
                    "anio": "anio",
                    "fuel": "motor",
                    "potencia": "potencia",
                    "precio": "precio"
                  }
            
df_segunda_mano.columns=df_segunda_mano.columns.map(dicc_encabezados3)

In [12]:
df_segunda_mano.head()

,id_segunda_mano,version,kilometraje,tipo,anio,motor,potencia,precio,provincia
0,1,Nissan Qashqai 1.2 DIG-T Acenta 4x2,121649,Manual,2015,Gasolina,116,11480,Madrid
1,2,Nissan Juke 1.5dCi Visia,260000,Manual,2012,Diésel,110,6500,Madrid
2,3,Nissan Qashqai DIG-T 103kW (140CV) mHEV 4x2 Ac...,11290,Manual,2023,Electro/Gasolina,140,22000,Madrid
3,4,Nissan Juke 1.6 N-Connecta 4x2 112,46287,Manual,2019,Gasolina,113,11790,Madrid
4,5,Nissan Qashqai 1.6dCi Visia 4x2,178000,Manual,2018,Diésel,131,13900,Madrid


In [ ]:
df_segunda_mano["potencia"].unique()

In [23]:
df_segunda_mano["provincia"]= "Madrid"

In [24]:
df_segunda_mano.to_csv("../datos/datos_segunda_mano.csv")

Datos de segunda mano ya están formateados

INICIO CREACIÓN DE TABLAS (agregando)

ANIO

In [13]:
lista_viejo=df_segunda_mano["anio"].unique()
lista_nuevo=df_datos_modelos_nuevos["anio"].unique()
lista_seg=df_seguridad["anio"].unique()
lista_anios= []
lista_anios.extend(lista_seg)
lista_anios.extend(lista_nuevo)
lista_anios.extend(lista_viejo)


In [14]:
df_anios=pd.DataFrame(lista_anios)
df_anios.duplicated().sum()

np.int64(15)

In [15]:
df_anios.drop_duplicates(inplace=True)
df_anios.reset_index(drop=True, inplace=True)
df_anios.duplicated().sum()
df_anios.index=df_anios.index+1
df_anios.reset_index(inplace=True)
dicc_anios={
            "index": "id_anio",
            0 : "anio"}
df_anios.columns=df_anios.columns.map(dicc_anios)   
df_anios.head()   

,id_anio,anio
0,1,2011
1,2,2012
2,3,2013
3,4,2014
4,5,2015


MOTOR

In [16]:
lista_viejo=df_segunda_mano["motor"].unique()
lista_nuevo=df_datos_modelos_nuevos["motor"].unique()
lista_motores= []
lista_motores.extend(lista_nuevo)
lista_motores.extend(lista_viejo)

In [17]:
df_motores=pd.DataFrame(lista_motores)
df_motores.duplicated().sum()

np.int64(2)

In [18]:
df_motores.drop_duplicates(inplace=True)
df_motores.reset_index(drop=True, inplace=True)
df_motores.index=df_motores.index+1
df_motores.reset_index(inplace=True)
dicc_motores={
            "index": "id_motor",
            0 : "motor"}
df_motores.columns=df_motores.columns.map(dicc_motores)   
df_motores.head()  

,id_motor,motor
0,1,MHEV Gasolina
1,2,HEV Gasolina
2,3,Gasolina
3,4,Eléctrico
4,5,Diesel


POTENCIA

In [19]:
lista_viejo=df_segunda_mano["potencia"].unique()
lista_nuevo=df_datos_modelos_nuevos["potencia"].unique()
lista_potencias= []
lista_potencias.extend(lista_nuevo)
lista_potencias.extend(lista_viejo)

In [20]:
df_potencias=pd.DataFrame(lista_potencias)
df_potencias.duplicated().sum()

np.int64(10)

In [21]:
df_potencias.drop_duplicates(inplace=True)
df_potencias.reset_index(drop=True, inplace=True)
df_potencias.index=df_potencias.index+1
df_potencias.reset_index(inplace=True)
dicc_potencias={
            "index": "id_potencia",
            0 : "potencia"}
df_potencias.columns=df_potencias.columns.map(dicc_potencias)   
df_potencias.head()

,id_potencia,potencia
0,1,140
1,2,158
2,3,190
3,4,114
4,5,143


EQUIPAMIENTOS SEGURIDAD

In [22]:
lista_seg1=df_seguridad["estabilidad_electronica"].unique()
lista_seg2=df_seguridad["aviso_choque_frontal"].unique()
lista_seg3=df_seguridad["aviso_desvio_carril"].unique()
lista_equipamientos= []
lista_equipamientos.extend(lista_seg1)
lista_equipamientos.extend(lista_seg2)
lista_equipamientos.extend(lista_seg3)

In [23]:
df_equipamientos=pd.DataFrame(lista_equipamientos)
df_equipamientos.duplicated().sum()

np.int64(6)

In [24]:
df_equipamientos.drop_duplicates(inplace=True)
df_equipamientos.reset_index(drop=True, inplace=True)
df_equipamientos.index=df_equipamientos.index+1
df_equipamientos.reset_index(inplace=True)
dicc_equipamientos={
            "index": "id_equipamiento",
            0 : "equipamiento"}
df_equipamientos.columns=df_equipamientos.columns.map(dicc_equipamientos)   
df_equipamientos

,id_equipamiento,equipamiento
0,1,Standard
1,2,Optional
2,3,No
3,4,Standard & Optional


CONTINUACION CREACION DE TABLAS

PROVINCIA

In [25]:
df_provincias= pd.DataFrame(df_datos_modelos_nuevos["provincia"].unique())
df_provincias.index=df_provincias.index+1
df_provincias.reset_index(inplace=True)

In [26]:
dicc_provincias={
            "index": "id_provincia",
            0 : "provincia"}
df_provincias.columns=df_provincias.columns.map(dicc_provincias)  
df_provincias

,id_provincia,provincia
0,1,Madrid


TIPO

In [27]:
df_cambio_marcha= pd.DataFrame(df_segunda_mano["tipo"].unique())
df_cambio_marcha.index=df_cambio_marcha.index+1
df_cambio_marcha.reset_index(inplace=True)

In [28]:
dicc_tipos={
            "index": "id_cambio",
            0 : "tipo"}
df_cambio_marcha.columns=df_cambio_marcha.columns.map(dicc_tipos)  
df_cambio_marcha

,id_cambio,tipo
0,1,Manual
1,2,Automático


MODELO

In [29]:
df_modelos= pd.DataFrame(df_datos_modelos_nuevos["modelo"].unique())
df_modelos.index=df_modelos.index+1
df_modelos.reset_index(inplace=True)

In [30]:
dicc_modelos={
            "index": "id_modelo",
            0 : "modelo"}
df_modelos.columns=df_modelos.columns.map(dicc_modelos)  
df_modelos.head()

,id_modelo,modelo
0,1,Nissan Qashqai
1,2,Nissan Qashqai híbrido
2,3,Nissan Juke
3,4,Nissan Juke híbrido
4,5,Nissan X-Trail


AHORA TOCA SUSTITUIR LOS IDS EN LAS TABLAS GRANDES

In [31]:
df_segunda_mano.head(3)

,id_segunda_mano,version,kilometraje,tipo,anio,motor,potencia,precio,provincia
0,1,Nissan Qashqai 1.2 DIG-T Acenta 4x2,121649,Manual,2015,Gasolina,116,11480,Madrid
1,2,Nissan Juke 1.5dCi Visia,260000,Manual,2012,Diésel,110,6500,Madrid
2,3,Nissan Qashqai DIG-T 103kW (140CV) mHEV 4x2 Ac...,11290,Manual,2023,Electro/Gasolina,140,22000,Madrid


In [32]:
df_datos_modelos_nuevos.head(3)

,id_modelo_nuevo,version,puertas,potencia,consumo,motor,precio,modelo,provincia,anio
0,1,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,5,140,6.3,MHEV Gasolina,31200,Nissan Qashqai,Madrid,2024
1,2,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,5,140,6.3,MHEV Gasolina,32700,Nissan Qashqai,Madrid,2024
2,3,NISSAN QASHQAI 1.3 DIG-T MHEV 116KW (158CV) AC...,5,158,6.3,MHEV Gasolina,34300,Nissan Qashqai,Madrid,2024


In [33]:
df_seguridad.head(3)

,id_seguridad,evaluacion_general,evaluacion_choque_frontal,evaluacion_choque_lateral,evaluacion_barrera_lateral,evaluacion_vuelco,probabilidad_vuelco,evaluacion_choque_columna,estabilidad_electronica,aviso_choque_frontal,aviso_desvio_carril,quejas,retiradas,investigaciones,anio,marca,version,descripcion,vehiculo_id
0,1,4,4,5,4,4,0.105,5,Standard,No,No,236,1,0,2011,NISSAN,ALTIMA,2011 Nissan Altima 4 DR FWD,357
1,2,4,4,5,4,4,0.105,5,Standard,No,No,240,1,1,2011,NISSAN,ALTIMA HEV,2011 Nissan Altima HEV 4 DR FWD,359
2,3,5,4,5,5,4,0.110,4,Standard,No,No,93,0,3,2011,NISSAN,LEAF,2011 Nissan Leaf 5 HB FWD,367


In [34]:
df_wallapop.head(3)

,id_wallapop,descripcion,precio,latitud,longitud,ciudad,provincia
0,1,nisan almera tino del 2004 ba por cadena le ve...,2200,40.301646,-3.441828,Arganda del Rey,Madrid
1,2,vendo mi coche en perfecto estado de motor y d...,6500,40.620256,-3.910636,Hoyo de Manzanares,Madrid
2,3,coche usado por mi mujer para ir a trabajar.\n...,11500,40.494190,-3.961537,Villanueva del Pardillo,Madrid


Primero en wallapop

In [35]:
dicc_provincia = df_provincias.set_index('provincia')['id_provincia'].to_dict()
dicc_provincia

{'Madrid': 1}

In [36]:
df_wallapop['id_provincia'] = df_wallapop['provincia'].map(dicc_provincia)


In [37]:
df_wallapop.drop(columns="provincia", inplace=True)

In [38]:
df_wallapop.head(2)

,id_wallapop,descripcion,precio,latitud,longitud,ciudad,id_provincia
0,1,nisan almera tino del 2004 ba por cadena le ve...,2200,40.301646,-3.441828,Arganda del Rey,1
1,2,vendo mi coche en perfecto estado de motor y d...,6500,40.620256,-3.910636,Hoyo de Manzanares,1


Segundo en nuevos

In [39]:
dicc_potencia = df_potencias.set_index('potencia')['id_potencia'].to_dict()
dicc_motor = df_motores.set_index('motor')['id_motor'].to_dict()
dicc_modelo = df_modelos.set_index('modelo')['id_modelo'].to_dict()
dicc_anio = df_anios.set_index('anio')['id_anio'].to_dict()

In [40]:
df_datos_modelos_nuevos['id_provincia'] = df_datos_modelos_nuevos['provincia'].map(dicc_provincia)
df_datos_modelos_nuevos['id_potencia'] = df_datos_modelos_nuevos['potencia'].map(dicc_potencia)
df_datos_modelos_nuevos['id_motor'] = df_datos_modelos_nuevos['motor'].map(dicc_motor)
df_datos_modelos_nuevos['id_modelo'] = df_datos_modelos_nuevos['modelo'].map(dicc_modelo)
df_datos_modelos_nuevos['id_anio'] = df_datos_modelos_nuevos['anio'].map(dicc_anio)


In [41]:
df_datos_modelos_nuevos.drop(columns="provincia", inplace=True)
df_datos_modelos_nuevos.drop(columns="potencia", inplace=True)
df_datos_modelos_nuevos.drop(columns="motor", inplace=True)
df_datos_modelos_nuevos.drop(columns="modelo", inplace=True)
df_datos_modelos_nuevos.drop(columns="anio", inplace=True)

In [42]:
df_datos_modelos_nuevos.head(2)

,id_modelo_nuevo,version,puertas,consumo,precio,id_provincia,id_potencia,id_motor,id_modelo,id_anio
0,1,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,5,6.3,31200,1,1,1,1,14
1,2,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,5,6.3,32700,1,1,1,1,14


Tercero en segunda mano

In [43]:
dicc_cambio = df_cambio_marcha.set_index('tipo')['id_cambio'].to_dict()

In [44]:
df_segunda_mano['id_provincia'] = df_segunda_mano['provincia'].map(dicc_provincia)
df_segunda_mano['id_potencia'] = df_segunda_mano['potencia'].map(dicc_potencia)
df_segunda_mano['id_motor'] = df_segunda_mano['motor'].map(dicc_motor)
df_segunda_mano['id_cambio'] = df_segunda_mano['tipo'].map(dicc_cambio)
df_segunda_mano['id_anio'] = df_segunda_mano['anio'].map(dicc_anio)


In [45]:
df_segunda_mano.drop(columns="provincia", inplace=True)
df_segunda_mano.drop(columns="potencia", inplace=True)
df_segunda_mano.drop(columns="motor", inplace=True)
df_segunda_mano.drop(columns="tipo", inplace=True)
df_segunda_mano.drop(columns="anio", inplace=True)

In [46]:
df_segunda_mano.head(2)

,id_segunda_mano,version,kilometraje,precio,id_provincia,id_potencia,id_motor,id_cambio,id_anio
0,1,Nissan Qashqai 1.2 DIG-T Acenta 4x2,121649,11480,1,18,3,1,5
1,2,Nissan Juke 1.5dCi Visia,260000,6500,1,15,6,1,2


Por útimo la Seguridad

In [47]:
dicc_equipamiento = df_equipamientos.set_index('equipamiento')['id_equipamiento'].to_dict()

In [48]:
df_seguridad['id_anio'] = df_seguridad['anio'].map(dicc_anio)
df_seguridad['id_equipamiento_aviso_choque_front'] = df_seguridad['aviso_choque_frontal'].map(dicc_equipamiento)
df_seguridad['id_equipamiento_aviso_desvio_carril'] = df_seguridad['aviso_desvio_carril'].map(dicc_equipamiento)
df_seguridad['id_equipamiento_estabilidad_electronica'] = df_seguridad['estabilidad_electronica'].map(dicc_equipamiento)

In [49]:
df_seguridad.drop(columns="anio", inplace=True)
df_seguridad.drop(columns="estabilidad_electronica", inplace=True)
df_seguridad.drop(columns="aviso_choque_frontal", inplace=True)
df_seguridad.drop(columns="aviso_desvio_carril", inplace=True)

In [50]:
df_seguridad.head(2)

,id_seguridad,evaluacion_general,evaluacion_choque_frontal,evaluacion_choque_lateral,evaluacion_barrera_lateral,evaluacion_vuelco,probabilidad_vuelco,evaluacion_choque_columna,quejas,retiradas,investigaciones,marca,version,descripcion,vehiculo_id,id_anio,id_equipamiento_aviso_choque_front,id_equipamiento_aviso_desvio_carril,id_equipamiento_estabilidad_electronica
0,1,4,4,5,4,4,0.105,5,236,1,0,NISSAN,ALTIMA,2011 Nissan Altima 4 DR FWD,357,1,3,3,1
1,2,4,4,5,4,4,0.105,5,240,1,1,NISSAN,ALTIMA HEV,2011 Nissan Altima HEV 4 DR FWD,359,1,3,3,1


In [56]:
df_segunda_mano.to_csv("../datos/datos_segunda_mano.csv")
df_seguridad.to_csv("../datos/datos_seguridad.csv")
df_datos_modelos_nuevos.to_csv("../datos/datos_modelos_nuevos.csv")
df_wallapop.to_csv("../datos/datos_wallapop.csv")

AHORA HABRÍA QUE CREAR LA BASE DE DATOS Y SUS RESPECTIVAS TABLAS

In [51]:
key= os.getenv("key")

In [176]:

conexion = psycopg2.connect(
    host="localhost",       
    user="postgres",      
    password=key, 
    port="5432"            
)


conexion.autocommit = True  
cursor = conexion.cursor()

nombre_base_datos = "Vehiculos"

try:
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(nombre_base_datos)))
except psycopg2.Error as e:
    print("Error de tipo:", e)





Error de tipo: la base de datos «Vehiculos» ya existe



In [177]:
try:
    conexion= psycopg2.connect(
        database= "Vehiculos",
        user= "postgres",
        password= key,
        host= "localhost",
        port= "5432"
    )
except:
    print(f"Ocurrió el error {e}")

AHORA LAS TABLAS

MODELOS

In [76]:
df_modelos.head(2)

,id_modelo,modelo
0,1,Nissan Qashqai
1,2,Nissan Qashqai híbrido


In [77]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS modelos(
    id_modelo INT PRIMARY KEY,
    modelo VARCHAR (150)
);

"""


cursor.execute(query_creacion)   
conexion.commit()


In [78]:
lista_de_tuplas=[]
for fila in df_modelos.values:
    lista_de_tuplas.append(tuple(fila))

In [79]:
     
query_insercion= "INSERT INTO modelos (id_modelo, modelo) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

TIPO

In [80]:
df_cambio_marcha.head(2)

,id_cambio,tipo
0,1,Manual
1,2,Automático


In [81]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS cambio_marcha(
    id_cambio INT PRIMARY KEY,
    tipo VARCHAR (100)
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [82]:


lista_de_tuplas=[]
for fila in df_cambio_marcha.values:
    lista_de_tuplas.append(tuple(fila))


In [83]:
     
query_insercion= "INSERT INTO cambio_marcha (id_cambio, tipo) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

Motor

In [84]:
df_motores.head(2)

,id_motor,motor
0,1,MHEV Gasolina
1,2,HEV Gasolina


In [85]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS motores(
    id_motor INT PRIMARY KEY,
    motor VARCHAR (100)
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [86]:


lista_de_tuplas=[]
for fila in df_motores.values:
    lista_de_tuplas.append(tuple(fila))


In [87]:
     
query_insercion= "INSERT INTO motores (id_motor, motor) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

POTENCIA

In [88]:
df_potencias.head(2)

,id_potencia,potencia
0,1,140
1,2,158


In [99]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS potencias(
    id_potencia INT PRIMARY KEY,
    potencia INT
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [100]:


lista_de_tuplas=[]
for fila in df_potencias.values:
    lista_de_tuplas.append(tuple(fila))


In [101]:
tuplas_int = [tuple(int(valor) for valor in tupla) for tupla in lista_de_tuplas]

In [102]:
     
query_insercion= "INSERT INTO potencias (id_potencia, potencia) VALUES (%s, %s);"     

cursor.executemany(query_insercion, tuplas_int)          
conexion.commit()

ANIO

In [103]:
df_anios.head(2)

,id_anio,anio
0,1,2011
1,2,2012


In [120]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS anios(
    id_anio INT PRIMARY KEY,
    anio INT
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [121]:


lista_de_tuplas=[]
for fila in df_anios.values:
    lista_de_tuplas.append(tuple(fila))
tuplas_int = [tuple(int(valor) for valor in tupla) for tupla in lista_de_tuplas]

In [122]:
     
query_insercion= "INSERT INTO anios (id_anio, anio) VALUES (%s, %s);"     

cursor.executemany(query_insercion, tuplas_int)          
conexion.commit()

PROVINCIAS

In [123]:
df_provincias.head(1)

,id_provincia,provincia
0,1,Madrid


In [124]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS provincias(
    id_provincia INT PRIMARY KEY,
    provincia VARCHAR (100)
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [125]:


lista_de_tuplas=[]
for fila in df_provincias.values:
    lista_de_tuplas.append(tuple(fila))


In [126]:
     
query_insercion= "INSERT INTO provincias (id_provincia, provincia) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

EQUIPAMIENTOS

In [127]:
df_equipamientos.head(2)

,id_equipamiento,equipamiento
0,1,Standard
1,2,Optional


In [128]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS equipamientos(
    id_equipamiento INT PRIMARY KEY,
    equipamiento VARCHAR (100)
);

"""
cursor.execute(query_creacion)   
conexion.commit()

In [129]:


lista_de_tuplas=[]
for fila in df_equipamientos.values:
    lista_de_tuplas.append(tuple(fila))


In [130]:
     
query_insercion= "INSERT INTO equipamientos (id_equipamiento, equipamiento) VALUES (%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

AHORA TOCAN LAS GRANDES

WALLAPOP

In [150]:
df_wallapop.head(2)

,id_wallapop,descripcion,precio,latitud,longitud,ciudad,id_provincia
0,1,nisan almera tino del 2004 ba por cadena le ve...,2200,40.301646,-3.441828,Arganda del Rey,1
1,2,vendo mi coche en perfecto estado de motor y d...,6500,40.620256,-3.910636,Hoyo de Manzanares,1


In [156]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS wallapop(
    id_wallapop INT PRIMARY KEY,
    descripcion VARCHAR (1000),
    precio INT,
    latitud NUMERIC,
    longitud NUMERIC,
    ciudad VARCHAR(200),
    id_provincia INT,
    FOREIGN KEY (id_provincia) REFERENCES provincias(id_provincia) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [157]:


lista_de_tuplas=[]
for fila in df_wallapop.values:
    lista_de_tuplas.append(tuple(fila))


In [158]:
     
query_insercion= "INSERT INTO wallapop (id_wallapop,descripcion,precio,latitud, longitud, ciudad,id_provincia ) VALUES (%s, %s,%s, %s,%s, %s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

SEGURIDAD

In [159]:
df_seguridad.head(2)

,id_seguridad,evaluacion_general,evaluacion_choque_frontal,evaluacion_choque_lateral,evaluacion_barrera_lateral,evaluacion_vuelco,probabilidad_vuelco,evaluacion_choque_columna,quejas,retiradas,investigaciones,marca,version,descripcion,vehiculo_id,id_anio,id_equipamiento_aviso_choque_front,id_equipamiento_aviso_desvio_carril,id_equipamiento_estabilidad_electronica
0,1,4,4,5,4,4,0.105,5,236,1,0,NISSAN,ALTIMA,2011 Nissan Altima 4 DR FWD,357,1,3,3,1
1,2,4,4,5,4,4,0.105,5,240,1,1,NISSAN,ALTIMA HEV,2011 Nissan Altima HEV 4 DR FWD,359,1,3,3,1


In [160]:
df_equipamientos.head(2)

,id_equipamiento,equipamiento
0,1,Standard
1,2,Optional


In [161]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS seguridad(
    id_seguridad INT PRIMARY KEY,
    evaluacion_general INT,
    evaluacion_choque_frontal INT,
    evaluacion_choque_lateral INT,
    evaluacion_barrera_lateral INT,
    evaluacion_vuelco INT,
    probabilidad_vuelco NUMERIC,
    evaluacion_choque_columna INT,
    quejas INT,
    retiradas INT,
    investigaciones INT,
    marca VARCHAR (100),
    version VARCHAR (200),
    descripcion VARCHAR (300),
    vehiculo_id INT,
    id_anio INT,
    id_equipamiento_aviso_choque_front INT,
    id_equipamiento_aviso_desvio_carril INT,
    id_equipamiento_estabilidad_electronica INT,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_equipamiento_aviso_choque_front) REFERENCES equipamientos(id_equipamiento) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_equipamiento_aviso_desvio_carril) REFERENCES equipamientos(id_equipamiento) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_equipamiento_estabilidad_electronica) REFERENCES equipamientos(id_equipamiento) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [162]:


lista_de_tuplas=[]
for fila in df_seguridad.values:
    lista_de_tuplas.append(tuple(fila))


In [163]:
     
query_insercion= "INSERT INTO seguridad (id_seguridad,evaluacion_general,evaluacion_choque_frontal,evaluacion_choque_lateral,evaluacion_barrera_lateral,evaluacion_vuelco,probabilidad_vuelco,evaluacion_choque_columna,quejas,retiradas,investigaciones, marca, version,descripcion,vehiculo_id, id_anio,id_equipamiento_aviso_choque_front,id_equipamiento_aviso_desvio_carril,id_equipamiento_estabilidad_electronica) VALUES (%s, %s,%s, %s,%s, %s, %s,%s, %s,%s, %s,%s, %s, %s,%s, %s, %s,%s, %s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

SEGUNDA MANO

In [164]:
df_segunda_mano.head(2)

,id_segunda_mano,version,kilometraje,precio,id_provincia,id_potencia,id_motor,id_cambio,id_anio
0,1,Nissan Qashqai 1.2 DIG-T Acenta 4x2,121649,11480,1,18,3,1,5
1,2,Nissan Juke 1.5dCi Visia,260000,6500,1,15,6,1,2


In [172]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS segunda_mano(
    id_segunda_mano INT PRIMARY KEY,
    version VARCHAR(200),
    kilometraje INT,
    precio INT,
    id_provincia INT,
    id_potencia INT,
    id_motor INT,
    id_cambio INT,
    id_anio INT,
    FOREIGN KEY (id_provincia) REFERENCES provincias(id_provincia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_potencia) REFERENCES potencias(id_potencia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_motor) REFERENCES motores(id_motor) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_cambio) REFERENCES cambio_marcha(id_cambio) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [173]:


lista_de_tuplas=[]
for fila in df_segunda_mano.values:
    lista_de_tuplas.append(tuple(fila))


In [174]:
     
query_insercion= "INSERT INTO segunda_mano (id_segunda_mano,version,kilometraje,precio,id_provincia,id_potencia,id_motor,id_cambio,id_anio) VALUES (%s, %s,%s, %s,%s, %s, %s,%s,%s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()

NUEVOS

In [178]:
df_datos_modelos_nuevos.head(2)

,id_modelo_nuevo,version,puertas,consumo,precio,id_provincia,id_potencia,id_motor,id_modelo,id_anio
0,1,NISSAN QASHQAI 1.3 DIG-T MHEV 103KW (140CV) AC...,5,6.3,31200,1,1,1,1,14
1,2,NISSAN QASHQAI (O)1.3 DIG-T MHEV 103KW (140CV)...,5,6.3,32700,1,1,1,1,14


In [179]:
cursor=conexion.cursor()
query_creacion=""" 
CREATE TABLE IF NOT EXISTS modelos_nuevos(
    id_modelo_nuevo INT PRIMARY KEY,
    version VARCHAR(400),
    puertas INT,
    consumo NUMERIC,
    precio INT,
    id_provincia INT,
    id_potencia INT,
    id_motor INT,
    id_modelo INT,
    id_anio INT,
    FOREIGN KEY (id_provincia) REFERENCES provincias(id_provincia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_potencia) REFERENCES potencias(id_potencia) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_motor) REFERENCES motores(id_motor) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_modelo) REFERENCES modelos(id_modelo) ON DELETE RESTRICT ON UPDATE CASCADE,
    FOREIGN KEY (id_anio) REFERENCES anios(id_anio) ON DELETE RESTRICT ON UPDATE CASCADE
);
"""
cursor.execute(query_creacion)   
conexion.commit()

In [180]:


lista_de_tuplas=[]
for fila in df_datos_modelos_nuevos.values:
    lista_de_tuplas.append(tuple(fila))


In [181]:
     
query_insercion= "INSERT INTO modelos_nuevos (id_modelo_nuevo,version,puertas,consumo,precio,id_provincia,id_potencia,id_motor,id_modelo,id_anio) VALUES (%s, %s,%s, %s,%s, %s, %s,%s,%s,%s);"     

cursor.executemany(query_insercion, lista_de_tuplas)          
conexion.commit()